<a href="https://colab.research.google.com/github/urvashi-agrawal-dev/gpt/blob/main/Gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget /content/shakespeare.txt

/content/shakespeare.txt: Scheme missing.


In [2]:
with open('shakespeare.txt', 'r',encoding='utf-8') as f:
  text = f.read()

In [3]:
print("length of dataset in character:",len(text))

length of dataset in character: 94275


In [4]:
print(text[:1000])

THE SONNETS

by William Shakespeare

From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou contracted to thine own bright eyes,
Feed'st thy light's flame with self-substantial fuel,
Making a famine where abundance lies,
Thy self thy foe, to thy sweet self too cruel:
Thou that art now the world's fresh ornament,
And only herald to the gaudy spring,
Within thine own bud buriest thy content,
And tender churl mak'st waste in niggarding:
Pity the world, or else this glutton be,
To eat the world's due, by the grave and thee.

When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a tattered weed of small worth held:  
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say within thine own deep sunken eyes,
Were an all-eating shame, and thriftless prais

In [5]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)


 !'(),-.:;?ABCDEFGHIJKLMNOPRSTUVWYabcdefghijklmnopqrstuvwxyz
61


In [6]:
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
encode=lambda s:[stoi[c]for c in s]
decode = lambda l:''.join([itos[i] for i in l])
print(encode("hi there"))
print(decode(encode("hi there")))

[42, 43, 1, 54, 42, 39, 52, 39]
hi there


In [7]:
import torch
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([94275]) torch.int64
tensor([30, 19, 16,  1, 29, 26, 25, 25, 16, 30, 29,  0,  0, 36, 59,  1, 33, 43,
        46, 46, 43, 35, 47,  1, 29, 42, 35, 45, 39, 53, 50, 39, 35, 52, 39,  0,
         0, 17, 52, 49, 47,  1, 40, 35, 43, 52, 39, 53, 54,  1, 37, 52, 39, 35,
        54, 55, 52, 39, 53,  1, 57, 39,  1, 38, 39, 53, 43, 52, 39,  1, 43, 48,
        37, 52, 39, 35, 53, 39,  6,  0, 30, 42, 35, 54,  1, 54, 42, 39, 52, 39,
        36, 59,  1, 36, 39, 35, 55, 54, 59,  3, 53,  1, 52, 49, 53, 39,  1, 47,
        43, 41, 42, 54,  1, 48, 39, 56, 39, 52,  1, 38, 43, 39,  6,  0, 13, 55,
        54,  1, 35, 53,  1, 54, 42, 39,  1, 52, 43, 50, 39, 52,  1, 53, 42, 49,
        55, 46, 38,  1, 36, 59,  1, 54, 43, 47, 39,  1, 38, 39, 37, 39, 35, 53,
        39,  6,  0, 19, 43, 53,  1, 54, 39, 48, 38, 39, 52,  1, 42, 39, 43, 52,
         1, 47, 43, 41, 42, 54,  1, 36, 39, 35, 52,  1, 42, 43, 53,  1, 47, 39,
        47, 49, 52, 59,  9,  0, 13, 55, 54,  1, 54, 42, 49, 55,  1, 37, 49, 48,
        

In [8]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[:n]

In [9]:
block_size=8
train_data[:block_size+1]

tensor([30, 19, 16,  1, 29, 26, 25, 25, 16])

In [10]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
  context=x[:t+1]
  target=y[t]
  print(f"when input is{context} the target:{target}")

when input istensor([30]) the target:19
when input istensor([30, 19]) the target:16
when input istensor([30, 19, 16]) the target:1
when input istensor([30, 19, 16,  1]) the target:29
when input istensor([30, 19, 16,  1, 29]) the target:26
when input istensor([30, 19, 16,  1, 29, 26]) the target:25
when input istensor([30, 19, 16,  1, 29, 26, 25]) the target:25
when input istensor([30, 19, 16,  1, 29, 26, 25, 25]) the target:16


In [11]:
torch.manual_seed(1337)
batch_size=4
block_size=4
def get_batch(split):
  data=train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y
xb,yb=get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('----')
for b in range(batch_size):
  for t in range(block_size):
    context=xb[b,:t+1]
    target=yb[b,t]
    print(f"when input is {context.tolist()} the target: {target}")


inputs:
torch.Size([4, 4])
tensor([[57, 42, 39, 48],
        [43, 39, 53,  1],
        [59, 39, 53,  1],
        [54, 52, 55, 35]])
targets:
torch.Size([4, 4])
tensor([[42, 39, 48,  1],
        [39, 53,  1, 49],
        [39, 53,  1, 37],
        [52, 55, 35, 48]])
----
when input is [57] the target: 42
when input is [57, 42] the target: 39
when input is [57, 42, 39] the target: 48
when input is [57, 42, 39, 48] the target: 1
when input is [43] the target: 39
when input is [43, 39] the target: 53
when input is [43, 39, 53] the target: 1
when input is [43, 39, 53, 1] the target: 49
when input is [59] the target: 39
when input is [59, 39] the target: 53
when input is [59, 39, 53] the target: 1
when input is [59, 39, 53, 1] the target: 37
when input is [54] the target: 52
when input is [54, 52] the target: 55
when input is [54, 52, 55] the target: 35
when input is [54, 52, 55, 35] the target: 48


In [12]:
print(xb)

tensor([[57, 42, 39, 48],
        [43, 39, 53,  1],
        [59, 39, 53,  1],
        [54, 52, 55, 35]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
  def forward(self,idx,targets=None):
    logits=self.token_embedding_table(idx)
    if targets is None:
      loss=None
    else:
      B,T,C=logits.shape
      logits=logits.view(B*T,C)
      targets=targets.view(B*T)
      loss=F.cross_entropy(logits,targets)
    return logits,loss

  def generate(self,idx,max_new_tokens):
    for _ in range(max_new_tokens):
      logits,loss=self(idx) # Call forward with only idx
      logits=logits[:,-1,:]
      probs=F.softmax(logits,dim=-1)
      idx_next=torch.multinomial(probs,num_samples=1)
      idx=torch.cat((idx,idx_next),dim=1)
    return idx
#xb,yb=get_batch('train')
m= BigramLanguageModel(vocab_size)
logits,loss=m(xb,yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([16, 61])
tensor(5.1809, grad_fn=<NllLossBackward0>)

nCG(fzIl(DjioeRaEsg: J!',csqug?kHooxW;,Ltf
DxiNti-:GTsEBjNLzjEbiTsEe(EuoGbtJwlH'EUOSwx)GMcEHCVDiTEmV


In [14]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [15]:
 batch_size=32
 for steps in range(10000):
  xb,yb=get_batch('train')
  logits,loss=m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  print(loss.item())

Streaming output truncated to the last 5000 lines.
2.3860154151916504
2.392463207244873
2.424748659133911
2.580627679824829
2.4761571884155273
2.4476513862609863
2.497248411178589
2.5319108963012695
2.446423292160034
2.524120807647705
2.437962055206299
2.447298049926758
2.2670068740844727
2.500730276107788
2.4387099742889404
2.5947484970092773
2.412613868713379
2.3797829151153564
2.596886396408081
2.406644344329834
2.350095748901367
2.3746745586395264
2.569255828857422
2.415386915206909
2.566986560821533
2.328139543533325
2.4206414222717285
2.5378408432006836
2.556863307952881
2.4432196617126465
2.362074851989746
2.30660080909729
2.462453603744507
2.5046403408050537
2.509838581085205
2.501157522201538
2.3875410556793213
2.4131176471710205
2.5204861164093018
2.6305220127105713
2.6027755737304688
2.5821526050567627
2.5663788318634033
2.3783726692199707
2.513460397720337
2.251128911972046
2.4123380184173584
2.5670483112335205
2.3884716033935547
2.5926616191864014
2.4823927879333496
2.4638

In [16]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=500)[0].tolist()))


Fo I weror wing kaki)' be o,
Tie by wngofoofas l seg twh ar thazith.
Din'smed tun t g averor  t q;or f swaurealase loveveigll.

Th y'nouts aimaghl,

Tow,
Angr'Tainoued sed ys Th isouearepulosleras y
Thir aringat me wowhateine ncad if f the, outottaresqu fov'e sh thasenday n pe burnu sf tonssurs wishoedisuis aike tndgut din ld t wass mout ldur  pougs, wimef y areathas purtho be bemo he cedothoff r thoril as, hen mu w lldera lr puivernolofeno t whous l, hen inoun wnd;Dearey,DI nd:
PAnce bone tho l


In [17]:
torch.manual_seed(1337)
B,T,C=4,8,2 #batch,time,channels
x=torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
xbow=torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev=x[b,:t+1]
    xbow[b,t]=torch.mean(xprev,0)


In [19]:
wei= torch.tril(torch.ones(T,T))
wei=wei/wei.sum(1,keepdim=True)
xbow2=wei@x
torch.allclose(xbow,xbow2)

False

In [20]:
tril=torch.tril(torch.ones(T,T))
wei=torch.zeros((T,T))
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,dim=-1)
xbox3=wei @ x
torch.allclose(xbow,xbox3)

False

In [21]:
torch.manual_seed(1337)
B,T,C=4,8,32
x=torch.randn(B,T,C)
head_size=16
key=nn.Linear(C,head_size,bias=False)
query=nn.Linear(C,head_size,bias=False)
value=nn.Linear(C,head_size,bias=False)
k=key(x)
q=query(x)
v=value(x)
wei=q @ k.transpose(-2,-1)
tril=torch.tril(torch.ones(T,T))
#wei=torch.zeros((T,T))
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,dim=-1)
out=wei@v
out.shape

torch.Size([4, 8, 16])

In [22]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [23]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [28]:
tril=torch.tril(torch.ones(T,T))
wei=torch.zeros((T,T))
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,dim=-1)
xbow3=wei@x
torch.allclose(xbow,xbox3)

False

In [29]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [30]:
torch.manual_seed(42)
a=torch.tril(torch.ones(3,3))
a=a/torch.sum(a,1,keepdim=True)
b=torch.randint(0,10,(3,2)).float()
c=a@b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [31]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [32]:
k=torch.randn(B,T,head_size)
q=torch.randn(B,T,head_size)
wei=q @ k.transpose(-2,-1)

In [33]:
k.var()

tensor(0.9331)

In [34]:
q.var()

tensor(0.8879)

In [35]:
wei.var()

tensor(13.0401)

In [36]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5]),dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [37]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5]*8),dim=-1)

tensor([0.0241, 0.0178, 0.0294, 0.0178, 0.0359, 0.0241, 0.0178, 0.0294, 0.0178,
        0.0359, 0.0241, 0.0178, 0.0294, 0.0178, 0.0359, 0.0241, 0.0178, 0.0294,
        0.0178, 0.0359, 0.0241, 0.0178, 0.0294, 0.0178, 0.0359, 0.0241, 0.0178,
        0.0294, 0.0178, 0.0359, 0.0241, 0.0178, 0.0294, 0.0178, 0.0359, 0.0241,
        0.0178, 0.0294, 0.0178, 0.0359])

In [38]:
class BatchNorm1d:
  def __init__(self,dim,eps=1e-5,momentum=0.1):
    self.eps=eps

    self.gamna=torch.ones(dim)
    self.beta=torch.zeros(dim)

  def __call__(self,x):

      xmean=x.mean(0,keepdim=True)
      xvar=x.var(0,keepdim=True)
      xhat=(x-xmean)/torch.sqrt(xvar+self.eps)
      self.out=self.gamna*xhat+self.beta

      return  self.out

  def parameters(self):
    return[self.gamma,self.beta]
torch.manual_seed(1337)
module=BatchNorm1d(100)
x=torch.randn(32,100)
x=module(x)
x.shape

torch.Size([32, 100])

In [39]:
x[:,0].mean(),x[:,0].std()

(tensor(7.4506e-09), tensor(1.0000))

In [40]:
x[0,:].mean(),x[0,:].std()

(tensor(0.0411), tensor(1.0431))